https://www.med66.com/sitemap/yixuetiku/1.htm

In [1]:
import re
import os
import requests
import hashlib
import json
import time
import random
import string
import pandas as pd
from tqdm import tqdm
import codecs
from glob import glob
from bs4 import BeautifulSoup
from scrapy.selector import Selector
from urllib.parse import unquote
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
# 定义随机的useragent
from fake_useragent import UserAgent
ua = UserAgent()

# 设置代理ip
ipdata = pd.read_excel('D:/Intern_Project/代理ip的地址2000.xlsx')
ipdata.fillna("",inplace = True)

proxieslist = []

for i in ipdata.index.values:
    line = ipdata.loc[i,['HTTP']].to_dict()
    proxieslist.append(line)
    
# randomproxy = random.choice(proxieslist)

# 设置消息头
headers_pc = {
    'User-Agent': ua.random,
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7'
}

params = {
    'ie': 'UTF-8',
    'wd': 'Python'
}


In [3]:
def changeparams():
    headers_pc = {
    'User-Agent': ua.random,
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7'
    }
    randomproxy = random.choice(proxieslist)
    #  访问百度服务器，获取cookie
#     res = requests.get('https://www.med66.com/sitemap/yixuetiku/1.htm', headers=headers_pc, proxies= randomproxy,)
#     # 将cookieJar数据转化为字典，s
#     cookie_dict = requests.utils.dict_from_cookiejar(res.cookies)
    return headers_pc, randomproxy

In [4]:
def html_download(headers_pc, randomproxy, item, count):
    # pbar.set_description(item['term'])
    try:
        url = item['url']
        # title = item['term'].strip().replace('/', ' ')
        if os.path.exists(f"D:/Intern_Project/13_医学题库/详情页/{count}.html"):
# #         if os.path.exists(f"D:/Intern_Project/5_everydayhealth/health_html/{newterm}.html"):
# #             pbar.update(1)    
            return None 
#         time.sleep(random.randint(0, 5))
        r = requests.get(url, headers=headers_pc, proxies=randomproxy, timeout=10)
        encoding = r.apparent_encoding
        html = r.content.decode(encoding, "ignore")
        
        with open(f"D:/Intern_Project/13_医学题库/详情页/{count}.html", 'w+', encoding=encoding) as f:
            f.write(html)
        
        pbar.update(1)
    except Exception as e:
        print(e)
        headers_pc, randomproxy = changeparams()
        mislist.append(item)
        pbar.update(1)
    return None

In [5]:
def task_monitor(task_list, mt_pool):
    while len(task_list) > 0:
        tmp_task_list = []
        for future in task_list:
            if future.done():
                result = future.result()
            else:
                tmp_task_list.append(future)
        
        task_list = tmp_task_list

In [6]:
with open('D:/Intern_Project/13_医学题库/qa链接列表.json', 'r', encoding='utf-8') as f:
    url_list = json.load(f)

print(len(url_list))


116322


In [7]:

pbar = tqdm(total=len(url_list))

# results = []
mislist = []
all_craw_tasks = []
headers_pc, randomproxy = changeparams()
count = 0

with ThreadPoolExecutor(max_workers=10) as mt_pool:
    for item in url_list:
        count += 1
        tmp_task = mt_pool.submit(html_download, headers_pc, randomproxy, item, count)
        all_craw_tasks.append(tmp_task)
        if count % 100 == 0:
            time.sleep(2)
            headers_pc, randomproxy = changeparams()

    task_monitor(all_craw_tasks, mt_pool)

  4%|███                                                                        | 4753/116322 [54:44<24:47:23,  1.25it/s]

('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None))


  9%|██████▌                                                                 | 10567/116322 [1:18:34<13:38:25,  2.15it/s]

In [11]:
print(mislist)

[]
